In [1]:
import os
import time
import re
import bisect
from collections import OrderedDict
import numpy as np
import tensorflow as tf
import scipy.ndimage
import scipy.misc

import config
import misc
import tfutil
import train
import dataset

In [ ]:
##Age Progression
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'age_progression_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/6),:] ####
        latents = np.tile(latents, [6, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:2] = [0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[2:4] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[4:6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[6:8] = [1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1] #### 000051
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2] = labels[0:8]
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3] = labels[0:8]
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4] = labels[0:8]
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5] = labels[0:8]
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6] = labels[0:8]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blond hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2,16] = 0.8
        labels[int(latents.shape[0]/6):int(latents.shape[0]/6)*2,9] = 0.2
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3,16] = 0.6
        labels[int(latents.shape[0]/6)*2:int(latents.shape[0]/6)*3,9] = 0.4
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4,16] = 0.4
        labels[int(latents.shape[0]/6)*3:int(latents.shape[0]/6)*4,9] = 0.6
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5,16] = 0.2
        labels[int(latents.shape[0]/6)*4:int(latents.shape[0]/6)*5,9] = 0.8
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6,16] = 0
        labels[int(latents.shape[0]/6)*5:int(latents.shape[0]/6)*6,9] = 1
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[8,6];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=1000)

In [ ]:
##Age Comparison
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'age_comparison_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/4),:] ####
        latents = np.tile(latents, [4, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:2] = [0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[2:4] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[4:6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[6:8] = [1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1] #### 000051
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2] = labels[0:8]
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3] = labels[0:8]
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4] = labels[0:8]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blond hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2,16] = 0
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2,9] = 0
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3,16] = 1
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3,9] = 1
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4,16] = 0
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4,9] = 1
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[8,4];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=1000)

In [ ]:
##Hair Progression
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'hair_color_progression_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/5),:] ####
        latents = np.tile(latents, [5, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:2] = [0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[2:4] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[4:6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[6:8] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
        labels[int(latents.shape[0]/5):int(latents.shape[0]/5)*2] = labels[0:8]
        labels[int(latents.shape[0]/5)*2:int(latents.shape[0]/5)*3] = labels[0:8]
        labels[int(latents.shape[0]/5)*3:int(latents.shape[0]/5)*4] = labels[0:8]
        labels[int(latents.shape[0]/5)*4:int(latents.shape[0]/5)*5] = labels[0:8]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        labels[int(latents.shape[0]/5):int(latents.shape[0]/5)*2,4:7] = [1, 0, 0]
        labels[int(latents.shape[0]/5):int(latents.shape[0]/5)*2,9] = 0
        labels[int(latents.shape[0]/5)*2:int(latents.shape[0]/5)*3,4:7] = [0, 1, 0]
        labels[int(latents.shape[0]/5)*2:int(latents.shape[0]/5)*3,9] = 0
        labels[int(latents.shape[0]/5)*3:int(latents.shape[0]/5)*4,4:7] = [0, 0, 1]
        labels[int(latents.shape[0]/5)*3:int(latents.shape[0]/5)*4,9] = 0
        labels[int(latents.shape[0]/5)*4:int(latents.shape[0]/5)*5,4:7] = [0, 0, 0]
        labels[int(latents.shape[0]/5)*4:int(latents.shape[0]/5)*5,9] = 1
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[8,5];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=800)

In [4]:
##beard_progression
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'beard_progression_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/4),:] ####
        latents = np.tile(latents, [4, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:3] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[3:6] = [1,0,0,1,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0] #### 000051
        labels[6:9] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2] = labels[0:9]
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3] = labels[0:9]
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4] = labels[0:9]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        labels[0:9,12] = 1
        labels[0:9,0] = 0
        labels[0:9,8] = 0
        labels[0:9,11] = 0
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2,0] = 2
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2,12] = 0
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3,8] = 2
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3,12] = 0
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4,11] = 2
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4,12] = 0
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[9,4];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=400)

Initializing TensorFlow...
Loading network from "results/063-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-006467.pkl"...
Saving results to report_results/052-beard_progression_63_6467
Generating png 0 / 1...


In [ ]:
##eyeglasses, hat, bangs
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'eyeglasses_hat_bangs_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/4),:] ####
        latents = np.tile(latents, [4, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:2] = [0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[2:4] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[4:6] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[6:8] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2] = labels[0:8]
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3] = labels[0:8]
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4] = labels[0:8]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        labels[int(latents.shape[0]/4):int(latents.shape[0]/4)*2,3] = 1
        labels[int(latents.shape[0]/4)*2:int(latents.shape[0]/4)*3,7] = 1
        labels[int(latents.shape[0]/4)*3:int(latents.shape[0]/4)*4,15] = 1
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[8,4];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=600)

In [ ]:
##User Report
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'User Report_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        latents = latents[0:int(len(latents)/2),:] ####
        latents = np.tile(latents, [2, 1])
        labels = np.zeros([latents.shape[0], 40], np.float32)
        #labels[0] = [0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        #labels[0:3] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[0:3] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        #labels[1] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
        labels[3:6] = labels[0:3]
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels
        
        #labels[3:6,4] = 0
        #labels[3:6,5] = 1
        #labels[3:6,0] = 0
        labels[3:6,16] = 0
        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[3,2];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=1000)

In [11]:
##Random Images
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'Random_Images_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:3] = [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        labels[3:6] = [1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0] #### 000051
        labels[6:9] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
        labels[9:12] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
        labels[12:15] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels

        
        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[3,5];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=1001)

Initializing TensorFlow...
Loading network from "results/063-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-006467.pkl"...
Saving results to report_results/059-Random_Images_63_6467
Generating png 0 / 1...


In [35]:
## more random images
def generate_fake_images(labels ,run_id=63, snapshot=6467, grid_size=[1,1], num_pngs=1, image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    network_pkl = misc.locate_network_pkl(run_id, snapshot)
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    random_state = np.random.RandomState(random_seed)

    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, snapshot)

    result_subdir = misc.create_result_subdir('report_results', 'More_Random_Images_'+str(run_id)+'_'+str(snapshot))
    for png_idx in range(num_pngs):
        print('Generating png %d / %d...' % (png_idx, num_pngs))
        latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
        labels = np.zeros([latents.shape[0], 40], np.float32)
        labels[0:np.prod(grid_size)] = [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
        
        new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
        new_labels[:,0] = labels[:,0] #5_oclock_shadow
        new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
        new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
        new_labels[:,6] = labels[:,11] #brown_hair
        new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
        new_labels[:,10] = labels[:,20] #male
        new_labels[:,11] = labels[:,22] #mustache
        new_labels[:,12] = labels[:,24] #no_beard
        new_labels[:,13] = labels[:,30] #side_burns
        new_labels[:,14] = labels[:,32] #straigth_hair
        new_labels[:,15] = labels[:,35] #hat
        new_labels[:,16] = labels[:,39] #young
        labels = new_labels

        labels[0:int(latents.shape[0]/8),:] = [0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0] #female, wavy, bangs, gray hair, old
        labels[int(latents.shape[0]/8):int(latents.shape[0]/8)*2,:] = [1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1]  #male, 5 oclock shadow, bangs, wavy, blonde hair, young
        labels[int(latents.shape[0]/8)*2:int(latents.shape[0]/8)*3,:] = [1,0,0,0,1,0,0,1,2,0,1,0,0,0,1,0,1] #male, 5 oclock shadow, goatee, black hair, straight hair, eye glasses, young
        labels[int(latents.shape[0]/8)*3:int(latents.shape[0]/8)*4,:] = [0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1] #female, blonde hair, wavy hair, young
        labels[int(latents.shape[0]/8)*4:int(latents.shape[0]/8)*5,:] = [0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,0,1] #female, black hair, bangs, straight hair, young, eye glasses
        labels[int(latents.shape[0]/8)*5:int(latents.shape[0]/8)*6,:] = [0,0,1,0,0,0,0,0,0,1,1,2,0,0,0,0,0] #male, bald, gray hair, mustach, old
        labels[int(latents.shape[0]/8)*6:int(latents.shape[0]/8)*7,:] = [0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,1] #male, brown hair, goatee, straight hair, young
        labels[int(latents.shape[0]/8)*7:int(latents.shape[0]/8)*8,:] = [0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1]  #female, brown hair, straight hair, young

        images = Gs.run(latents, labels, minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, png_idx)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[9,8];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_id, snapshot=6467, grid_size = grid_size, random_seed=1002)

Initializing TensorFlow...
Loading network from "results/063-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-006467.pkl"...
Saving results to report_results/082-More_Random_Images_63_6467
Generating png 0 / 1...


In [2]:
## progression through training
def generate_fake_images(labels ,run_ids_snapshots=[[63,6467]], grid_size=[1,1], image_shrink=1, png_prefix=None, random_seed=1000, minibatch_size=8):
    random_state = np.random.RandomState(random_seed)
    result_subdir = misc.create_result_subdir('report_results', 'training_progression')
    images = np.zeros([np.prod(grid_size),3,64,64])
    
    network_pkl = misc.locate_network_pkl(run_ids_snapshots[0][0], run_ids_snapshots[0][1])
    if png_prefix is None:
        png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
    print('Loading network from "%s"...' % network_pkl)
    G, D, Gs = misc.load_network_pkl(network_pkl, run_ids_snapshots[0][1])
    
    latents = misc.random_latents(np.prod(grid_size), Gs, random_state=random_state) 
    latents = latents[0:int(len(latents)/8),:] ####
    latents = np.tile(latents, [8, 1])
    labels = np.zeros([latents.shape[0], 40], np.float32)
    labels[0] = [0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1] #### 000002
    labels[1] = [1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0] #### 000051
    labels[2] = [1,0,1,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1] #### 000007
    labels[3] = [0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1] #### 000006
    labels[4] = [0,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0] #### 000052
    labels[int(latents.shape[0]/8):int(latents.shape[0]/8)*2] = labels[0:5]
    labels[int(latents.shape[0]/8)*2:int(latents.shape[0]/8)*3] = labels[0:5]
    labels[int(latents.shape[0]/8)*3:int(latents.shape[0]/8)*4] = labels[0:5]
    labels[int(latents.shape[0]/8)*4:int(latents.shape[0]/8)*5] = labels[0:5]
    labels[int(latents.shape[0]/8)*5:int(latents.shape[0]/8)*6] = labels[0:5]
    labels[int(latents.shape[0]/8)*6:int(latents.shape[0]/8)*7] = labels[0:5]
    labels[int(latents.shape[0]/8)*7:int(latents.shape[0]/8)*8] = labels[0:5]
    
    new_labels = np.zeros((labels.shape[0],17), dtype=np.float32);
    new_labels[:,0] = labels[:,0] #5_oclock_shadow
    new_labels[:,1:4] = labels[:,3:6] #wavy_hair, Bald, Bangs
    new_labels[:,4:6] = labels[:,8:10] #black_hair, blonde hair
    new_labels[:,6] = labels[:,11] #brown_hair
    new_labels[:,7:10] = labels[:,15:18] #Eyeglasses, goatee, gray_hair
    new_labels[:,10] = labels[:,20] #male
    new_labels[:,11] = labels[:,22] #mustache
    new_labels[:,12] = labels[:,24] #no_beard
    new_labels[:,13] = labels[:,30] #side_burns
    new_labels[:,14] = labels[:,32] #straigth_hair
    new_labels[:,15] = labels[:,35] #hat
    new_labels[:,16] = labels[:,39] #young
    labels = new_labels
    
    images[0:8] = Gs.run(latents[0:8], labels[0:8], minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
    
    for i in range(1,8):
        network_pkl = misc.locate_network_pkl(run_ids_snapshots[i][0], run_ids_snapshots[i][1])
        if png_prefix is None:
            png_prefix = misc.get_id_string_for_network_pkl(network_pkl) + '-'
        print('Loading network from "%s"...' % network_pkl)
        G, D, Gs = misc.load_network_pkl(network_pkl, run_ids_snapshots[i][1])
        images[i*8:(i+1)*8] = Gs.run(latents[i*8:(i+1)*8], labels[i*8:(i+1)*8], minibatch_size=minibatch_size, num_gpus=config.num_gpus, out_mul=127.5, out_add=127.5, out_shrink=image_shrink, out_dtype=np.uint8)
        
    misc.save_image_grid(images, os.path.join(result_subdir, '%s%06d.png' % (png_prefix, 0)), [0,255], grid_size)
    open(os.path.join(result_subdir, '_done.txt'), 'wt').close()

run_id = 63
grid_size=[5,8];
num_pngs=1
#import ipdb; ipdb.set_trace() # debugging starts here
np.random.seed(config.random_seed)
labels = []
print('Initializing TensorFlow...')
os.environ.update(config.env)
tfutil.init_tf(config.tf_config)
generate_fake_images(labels, run_ids_snapshots = [[58,160],[58,1462],[58,2826],[58,3547],[58,4227],[58,4947],[60,5827],[63,6467]], grid_size = grid_size, random_seed=1300)

Initializing TensorFlow...
Saving results to report_results/087-training_progression
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-000160.pkl"...
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-001462.pkl"...
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-002826.pkl"...
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-003547.pkl"...
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-004227.pkl"...
Loading network from "results/058-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-004947.pkl"...
Loading network from "results/060-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-005827.pkl"...
Loading network from "results/063-pgan-celeba-cond-preset-v2-1gpu-fp32-GRAPH/network-snapshot-006467.pkl"...
